In [80]:
import pymysql
import requests

In [ ]:
#Se deben colocar loa datos correctos en la parte del codigo donde se genera la conexion con mysql, de lo contrario
#no funcionara la creacion y el volcado de datos de la base de datos

### Ingesta un CVS desde un link de descarga, se limpian y ordenan los datos ###

In [81]:
#saca el contenido del csv como un string, directo del link de descarga del .csv
URL_BASE = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

In [82]:
#quitas las comillas con las que ya viene el string
html_obtenido = html_obtenido.replace('"','')
#divide el string por linea y le quita salto de lineas y \r
linea = html_obtenido.replace('\r','').split('\n')

#separa los elementos que conforman cada linea
dentro=[]
for i in linea:
    dentro.append(i.split(','))

#la ultima linea de dentro esta vacia, por lo que se debe limpiar
dentro.pop(-1)    

['']

In [83]:
#saca los datos de nombre,categoria,barrio,comuna
nombre=[]
categoria=[]
barrio=[]
comuna=[]
for i in range(0,len(dentro)):
   nombre.append(dentro[i][3])
   categoria.append(dentro[i][4])
   barrio.append(dentro[i][-4])
   comuna.append(dentro[i][-3])

#cambia los vacios por 'NULL' en categoria
for i,l in enumerate(categoria):
    if l == '' :
        categoria[i] = 'NULL'
  

In [84]:
#para sacar el valor de direccion, porque tenia varias ',' dentro de los campos
direccion=[]
for i in range(0,len(dentro)):
   largo=len(dentro[i])
#si tiene len 19
   if largo == 19:
      direccion.append(dentro[i][13]+','+dentro[i][14])
#si tiene len 20
   elif largo == 20:
       direccion.append(dentro[i][13]+','+dentro[1][14] +','+dentro[i][15])
#si tiene len 18
   else:
       direccion.append(dentro[i][13])

In [85]:
#toca sacar los primeros elementos de las lista, porque no son los datos
#los guardo en una lista con los nombre de las columnas
columnas =[nombre.pop(0), categoria.pop(0), direccion.pop(0), barrio.pop(0), comuna.pop(0)]
columnas


['nombre', 'categoria', 'direccion_completa', 'barrio', 'comuna']

In [90]:
#transforma las listas con los datos a una lista de tuplas
pre_lista=[]
datos=[]

for i in range(0,len(nombre)): 
    pre_lista = [nombre[i], categoria[i], direccion[i], barrio[i], comuna[i]]
    datos.append(tuple(pre_lista))

### Crea una base de datos y sube los datos a una base de datos local de mysql ###

In [86]:
#Establece la conexion con el servidor local, sin especificar la base de datos
conexion = pymysql.connect( host='localhost', user='root', passwd='yourpassword')
#crea cursor
cursor = conexion.cursor()


In [87]:
#crea la base de datos
crear_base = """ CREATE DATABASE IF NOT EXISTS oferta_gastronomica; """
cursor.execute(crear_base)


1

In [88]:
#cierro y vuelvo a abrir la concexion ya dentro de la base de datos creada
#si no lo hago da un error, no me sirvio usar USE oferta_gastronomica;

conexion.close()
#Establece la conexion con el servidor local, ya dento de base de datos
conexion = pymysql.connect( host='localhost', user='root', passwd='yourpassword', db='oferta_gastronomica' ) 
#crea cursor
cursor = conexion.cursor()

In [89]:
#Crear tabla
crear_tabla = """
    CREATE TABLE  oferta (
	id_local INT NOT NULL AUTO_INCREMENT,
    nombre VARCHAR (40),
    categoria VARCHAR (40),
    direccion VARCHAR (80),
    barrio VARCHAR (40),
    comuna VARCHAR (40),
    PRIMARY KEY (id_local)
    );"""
cursor.execute(crear_tabla)


0

In [91]:
#insertar valores
cursor.executemany("INSERT INTO oferta (nombre, categoria, direccion, barrio, comuna) VALUES (%s, %s, %s, %s, %s)", datos)

2823

In [92]:
#para insertar los datos en la tabla
conexion.commit()
#cierra conexion
conexion.close()